In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!sudo pip install keras
!sudo pip install tensorflow

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab_Datas/시군구별전력사용량_202408.csv',encoding='cp949')

# 필요한 열 선택 및 조건 필터링
df = df[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월']]
df = df[df['계약종별'] == '합 계']

# 월별 전력 소비량을 숫자형으로 변환
for month in ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월']:
    df[month] = pd.to_numeric(df[month])

# 울산 지역의 데이터만 필터링
ulsan_data = df[df['시도'] == '울산광역시'].drop(columns=['시도', '계약종별'])
# 시군구 열을 인덱스로 설정하여 제거
ulsan_data.set_index('시군구', inplace=True)
data = ulsan_data.T  # 전력 소비량을 행으로 변환

# 데이터셋을 LSTM 입력 형식으로 변환하는 함수
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        a = data[i:(i + time_step), 0]
        X.append(a)
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# LSTM 학습 및 예측
predictions = []

# 울산 지역의 시군구에 대해 반복
for index in range(data.shape[1]):  # 각 시군구에 대해 반복
    current_sigu = ulsan_data.index[index]  # 시군구 이름을 가져옴
    print(f"현재 학습 중인 시군구: {current_sigu}")

    sigu_data = data.iloc[:, index].values.reshape(-1, 1)  # 시군구 데이터

    # 데이터셋을 LSTM 입력 형식으로 변환 (X, y)
    X, y = create_dataset(sigu_data, time_step=7)  # 1월부터 7월까지의 데이터 사용

    # 데이터 차원 조정
    if X.shape[0] > 0:  # X가 비어 있지 않을 때만 조정
        X = X.reshape(X.shape[0], X.shape[1], 1)

        # LSTM 모델 구축
        model = Sequential()
        model.add(LSTM(20, return_sequences=True, input_shape=(X.shape[1], 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(20, return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        # 모델 학습
        model.fit(X, y, epochs=30, batch_size=32)

        # 8월 값 예측을 위한 데이터 준비
        last_data = sigu_data[-7:].reshape(1, 7, 1)  # 7월까지의 데이터
        predicted_august = model.predict(last_data)
        predictions.append((current_sigu, predicted_august[0][0]))  # (시군구, 예측값)

# 예측 결과 출력
for sigu, pred in predictions:
    # 울산 지역의 실제 8월 값
    actual_august = ulsan_data.loc[sigu, '8월']  # 실제 8월 값
    #print(f"{sigu}의 Predicted 8월 전력 사용량: {pred}, Actual 8월 전력 사용량: {actual_august}, 차이: {actual_august - pred}")

import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# 정규화 함수 정의
def normalize(data):
    data_min = np.min(data)
    data_max = np.max(data)
    normalized_data = (data - data_min) / (data_max - data_min)
    return normalized_data, data_min, data_max

# LSTM 학습 및 예측
predictions = []

# 울산 지역의 시군구에 대해 반복
for index in range(data.shape[1]):  # 각 시군구에 대해 반복
    current_sigu = ulsan_data.index[index]  # 시군구 이름을 가져옴
    print(f"현재 학습 중인 시군구: {current_sigu}")

    sigu_data = data.iloc[:, index].values.reshape(-1, 1)  # 시군구 데이터

    # 데이터 정규화
    normalized_data, data_min, data_max = normalize(sigu_data)

    # 데이터셋을 LSTM 입력 형식으로 변환 (X, y)
    X, y = create_dataset(normalized_data, time_step=7)  # 1월부터 7월까지의 데이터 사용

    # 데이터 차원 조정
    if X.shape[0] > 0:  # X가 비어 있지 않을 때만 조정
        X = X.reshape(X.shape[0], X.shape[1], 1)

        # LSTM 모델 구축
        model = Sequential()
        model.add(LSTM(20, return_sequences=True, input_shape=(X.shape[1], 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(20, return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        # 모델 학습
        model.fit(X, y, epochs=30, batch_size=32)

        # 8월 값 예측을 위한 데이터 준비
        last_data = normalized_data[-7:].reshape(1, 7, 1)  # 7월까지의 데이터
        predicted_august = model.predict(last_data)

        # 예측값을 원래 전력 사용량으로 복원
        predicted_august_original = predicted_august * (data_max - data_min) + data_min
        predictions.append((current_sigu, predicted_august_original[0][0]))  # (시군구, 예측값)

# 예측 결과 출력
for sigu, pred in predictions:
    # 울산 지역의 실제 8월 값
    actual_august = ulsan_data.loc[sigu, '8월']  # 실제 8월 값

    # 예측값 대비 실제 사용량 비율 계산
    usage_percentage = (actual_august / pred) * 100 if pred != 0 else 0

    print(f"{sigu}의 Predicted 8월 전력 사용량: {pred:.2f}, "
          f"Actual 8월 전력 사용량: {actual_august}, "
          f"차이: {actual_august - pred:.2f}, "
          f"실제 사용량 비율: {usage_percentage:.2f}%")
